In [1]:
"""
CHAMPIONSHIP MODEL - Insurance Agent NILL Prediction
Data Storm v6.0 - First Place Solution (with Optuna HPO)

Key enhancements:
1. Stratified time-series cross-validation with gap
2. Feature importance-based selection with stability analysis
3. CatBoost integration with custom loss function
4. Agent-specific dynamic thresholding
5. Recursive feature elimination with stability scores
6. Optuna for Hyperparameter Optimization
"""
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
from sklearn.model_selection import TimeSeriesSplit, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.feature_selection import RFECV, SelectFromModel
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import optuna # Optuna import

import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
optuna.logging.set_verbosity(optuna.logging.INFO) # Optuna verbosity

# Get relative paths
try:
    script_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    script_dir = os.getcwd() # Fallback for interactive environments

data_dir = os.path.join(script_dir, 'dataset')
output_dir = os.path.join(script_dir, 'outputs')

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("=" * 100)
print("CHAMPIONSHIP KAGGLE SOLUTION - ADVANCED ENSEMBLE WITH CUSTOM AGENT PROFILING & OPTUNA HPO")
print("=" * 100)
start_time = time.time()
print(f"Starting at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Load data with integrity checks
print("\nStep 1: Loading data with enhanced checks...")
train_df = pd.read_csv(os.path.join(data_dir, 'train_storming_round.csv'))
test_df = pd.read_csv(os.path.join(data_dir, 'test_storming_round.csv'))
submission_template = pd.read_csv(os.path.join(data_dir, 'sample_submission_storming_round.csv'))

print(f"Train data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print(f"Submission template shape: {submission_template.shape}")

# Critical integrity checks and deduplications
print("Performing data integrity checks...")
assert len(test_df) == len(submission_template), "Test and submission sizes don't match!"

# Check for duplicates in train data
dupes_train = train_df.duplicated().sum()
if dupes_train > 0:
    print(f"WARNING: Found {dupes_train} duplicate rows in training data. Removing...")
    train_df = train_df.drop_duplicates().reset_index(drop=True)

# Check for duplicates in test data
dupes_test = test_df.duplicated().sum()
if dupes_test > 0:
    print(f"WARNING: Found {dupes_test} duplicate rows in test data. Removing...")
    test_df = test_df.drop_duplicates().reset_index(drop=True)

# Advanced preprocessing
print("\nStep 2: Enhanced preprocessing with domain expertise...")
date_columns = ['agent_join_month', 'first_policy_sold_month', 'year_month']
for df in [train_df, test_df]:
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col])

# Create better target variable (looking ahead one month)
train_df = train_df.sort_values(['agent_code', 'year_month'])
train_df['target_column'] = 0  # Default to 0 (will go NILL)

# Get unique agents and process each for sophisticated target creation
unique_agents = train_df['agent_code'].unique()
for agent in unique_agents:
    agent_data = train_df[train_df['agent_code'] == agent].copy()
    agent_data = agent_data.sort_values('year_month')

    # For each month, check if agent sells anything in the next month
    for i in range(len(agent_data) - 1):
        current_row_id = agent_data.iloc[i]['row_id']
        next_month_sales = agent_data.iloc[i+1]['new_policy_count']

        # If they sell anything next month, target is 1 (not NILL)
        if next_month_sales > 0:
            train_df.loc[train_df['row_id'] == current_row_id, 'target_column'] = 1

# Remove the last month record for each agent as we don't have next month data
last_month_indices = []
for agent in unique_agents:
    agent_data = train_df[train_df['agent_code'] == agent]
    if len(agent_data) > 0:  # Safety check
        last_month_idx = agent_data.iloc[-1].name
        last_month_indices.append(last_month_idx)

train_df = train_df.drop(last_month_indices)
print(f"Processed training data shape: {train_df.shape}")
print(f"Target distribution: {train_df['target_column'].value_counts()}")

# Calculate class imbalance for weighing
train_class_weights = {
    0: train_df.shape[0] / (2 * (train_df['target_column'] == 0).sum()),
    1: train_df.shape[0] / (2 * (train_df['target_column'] == 1).sum())
}
print(f"Class weights for imbalance handling: {train_class_weights}")

# Enhanced Feature Engineering
print("\nStep 3: Advanced feature engineering with agent profiling...")

# Process each dataframe separately to avoid duplication
for df in [train_df, test_df]:
    # Extract time-based features
    for col in date_columns:
        if col in df.columns:
            df[f'{col}_month'] = df[col].dt.month
            df[f'{col}_year'] = df[col].dt.year
            df[f'{col}_quarter'] = df[col].dt.quarter
            df[f'{col}_month_sin'] = np.sin(2 * np.pi * df[col].dt.month/12)
            df[f'{col}_month_cos'] = np.cos(2 * np.pi * df[col].dt.month/12)

    # Experience features
    if all(col in df.columns for col in ['year_month', 'agent_join_month']):
        df['months_with_company'] = ((df['year_month'].dt.year - df['agent_join_month'].dt.year) * 12 +
                                    (df['year_month'].dt.month - df['agent_join_month'].dt.month))

    if all(col in df.columns for col in ['first_policy_sold_month', 'agent_join_month']):
        df['months_to_first_sale'] = ((df['first_policy_sold_month'].dt.year - df['agent_join_month'].dt.year) * 12 +
                                    (df['first_policy_sold_month'].dt.month - df['agent_join_month'].dt.month))
        df['months_to_first_sale'] = df['months_to_first_sale'].fillna(-1)

    if all(col in df.columns for col in ['year_month', 'first_policy_sold_month']):
        df['months_since_first_sale'] = ((df['year_month'].dt.year - df['first_policy_sold_month'].dt.year) * 12 +
                                      (df['year_month'].dt.month - df['first_policy_sold_month'].dt.month))
        df['months_since_first_sale'] = df['months_since_first_sale'].fillna(-1)

    # Activity trend features
    if all(col in df.columns for col in ['unique_proposals_last_7_days', 'unique_proposals_last_15_days']):
        df['proposal_trend_7_15'] = df['unique_proposals_last_7_days'] / np.maximum(df['unique_proposals_last_15_days'], 1)
    if all(col in df.columns for col in ['unique_proposals_last_15_days', 'unique_proposals_last_21_days']):
        df['proposal_trend_15_21'] = df['unique_proposals_last_15_days'] / np.maximum(df['unique_proposals_last_21_days'], 1)
    if all(col in df.columns for col in ['unique_quotations_last_7_days', 'unique_quotations_last_15_days']):
        df['quotation_trend_7_15'] = df['unique_quotations_last_7_days'] / np.maximum(df['unique_quotations_last_15_days'], 1)
    if all(col in df.columns for col in ['unique_quotations_last_15_days', 'unique_quotations_last_21_days']):
        df['quotation_trend_15_21'] = df['unique_quotations_last_15_days'] / np.maximum(df['unique_quotations_last_21_days'], 1)
    if all(col in df.columns for col in ['unique_customers_last_7_days', 'unique_customers_last_15_days']):
        df['customer_trend_7_15'] = df['unique_customers_last_7_days'] / np.maximum(df['unique_customers_last_15_days'], 1)
    if all(col in df.columns for col in ['unique_customers_last_15_days', 'unique_customers_last_21_days']):
        df['customer_trend_15_21'] = df['unique_customers_last_15_days'] / np.maximum(df['unique_customers_last_21_days'], 1)

    # Activity consistency (variance-based)
    if all(col in df.columns for col in ['unique_proposals_last_7_days', 'unique_proposals_last_15_days', 'unique_proposals_last_21_days']):
        proposal_cols = ['unique_proposals_last_7_days', 'unique_proposals_last_15_days', 'unique_proposals_last_21_days']
        df['proposal_variance'] = df[proposal_cols].var(axis=1)
        df['proposal_consistency'] = 1 / (1 + df['proposal_variance'])
    if all(col in df.columns for col in ['unique_quotations_last_7_days', 'unique_quotations_last_15_days', 'unique_quotations_last_21_days']):
        quotation_cols = ['unique_quotations_last_7_days', 'unique_quotations_last_15_days', 'unique_quotations_last_21_days']
        df['quotation_variance'] = df[quotation_cols].var(axis=1)
        df['quotation_consistency'] = 1 / (1 + df['quotation_variance'])
    if all(col in df.columns for col in ['unique_customers_last_7_days', 'unique_customers_last_15_days', 'unique_customers_last_21_days']):
        customer_cols = ['unique_customers_last_7_days', 'unique_customers_last_15_days', 'unique_customers_last_21_days']
        df['customer_variance'] = df[customer_cols].var(axis=1)
        df['customer_consistency'] = 1 / (1 + df['customer_variance'])

    # Current period activity rates
    if all(col in df.columns for col in ['unique_customers', 'unique_proposal']):
        df['proposals_per_customer'] = df['unique_proposal'] / np.maximum(df['unique_customers'], 1)
    if all(col in df.columns for col in ['unique_customers', 'unique_quotations']):
        df['quotations_per_customer'] = df['unique_quotations'] / np.maximum(df['unique_customers'], 1)
    if all(col in df.columns for col in ['unique_proposal', 'unique_quotations']):
        df['quotations_per_proposal'] = df['unique_quotations'] / np.maximum(df['unique_proposal'], 1)

    # Time-based seasonality features
    if 'year_month_month' in df.columns:
        df['is_quarter_end'] = df['year_month_month'].isin([3, 6, 9, 12]).astype(int)
        df['is_year_end'] = df['year_month_month'].isin([12]).astype(int)

    # Ratios of activity metrics
    if all(col in df.columns for col in ['unique_proposal', 'unique_quotations']):
        df['quotation_to_proposal_ratio'] = df['unique_quotations'] / np.maximum(df['unique_proposal'], 1)

    # Cash payment ratio
    if all(col in df.columns for col in ['number_of_cash_payment_policies', 'number_of_policy_holders']):
        df['cash_payment_ratio'] = df['number_of_cash_payment_policies'] / np.maximum(df['number_of_policy_holders'], 1)

    # Agent characteristics
    if 'agent_age' in df.columns:
        df['agent_age_squared'] = df['agent_age'] ** 2
        df['agent_age_prime'] = (df['agent_age'] >= 35) & (df['agent_age'] <= 45)
        df['agent_senior'] = df['agent_age'] > 50

    # Interaction features
    if all(col in df.columns for col in ['agent_age', 'months_with_company']):
        df['age_experience_interaction'] = df['agent_age'] * df['months_with_company']
    if all(col in df.columns for col in ['agent_age', 'months_since_first_sale']):
        df['age_sales_experience'] = df['agent_age'] * np.maximum(df['months_since_first_sale'], 0)

    # Agent velocity metrics
    if all(col in df.columns for col in ['unique_proposals_last_7_days', 'unique_proposal']):
        df['proposal_velocity'] = df['unique_proposals_last_7_days'] / np.maximum(df['unique_proposal'], 1)
    if all(col in df.columns for col in ['unique_quotations_last_7_days', 'unique_quotations']):
        df['quotation_velocity'] = df['unique_quotations_last_7_days'] / np.maximum(df['unique_quotations'], 1)
    if all(col in df.columns for col in ['unique_customers_last_7_days', 'unique_customers']):
        df['customer_velocity'] = df['unique_customers_last_7_days'] / np.maximum(df['unique_customers'], 1)

    # CHAMPION-SPECIFIC FEATURES
    if all(col in df.columns for col in ['unique_proposals_last_7_days', 'unique_proposals_last_21_days']):
        df['proposal_momentum'] = df['unique_proposals_last_7_days'] / np.maximum(df['unique_proposals_last_21_days'], 1) * 3
    if all(col in df.columns for col in ['unique_quotations_last_7_days', 'unique_quotations_last_21_days']):
        df['quotation_momentum'] = df['unique_quotations_last_7_days'] / np.maximum(df['unique_quotations_last_21_days'], 1) * 3
    if all(col in df.columns for col in ['unique_customers_last_7_days', 'unique_customers_last_21_days']):
        df['customer_momentum'] = df['unique_customers_last_7_days'] / np.maximum(df['unique_customers_last_21_days'], 1) * 3

    if all(col in df.columns for col in ['unique_proposal', 'unique_proposals_last_7_days',
                                        'unique_proposals_last_15_days', 'unique_proposals_last_21_days']):
        df['proposal_gap'] = df['unique_proposal'] - (df['unique_proposals_last_7_days'] +
                                                  df['unique_proposals_last_15_days'] +
                                                  df['unique_proposals_last_21_days'])
    if all(col in df.columns for col in ['unique_quotations', 'unique_quotations_last_7_days',
                                        'unique_quotations_last_15_days', 'unique_quotations_last_21_days']):
        df['quotation_gap'] = df['unique_quotations'] - (df['unique_quotations_last_7_days'] +
                                                     df['unique_quotations_last_15_days'] +
                                                     df['unique_quotations_last_21_days'])
    if all(col in df.columns for col in ['ANBP_value', 'unique_proposal']):
        df['revenue_per_proposal'] = df['ANBP_value'] / np.maximum(df['unique_proposal'], 1)
    if all(col in df.columns for col in ['ANBP_value', 'unique_customers']):
        df['revenue_per_customer'] = df['ANBP_value'] / np.maximum(df['unique_customers'], 1)

    if all(col in df.columns for col in ['new_policy_count', 'unique_proposal']):
        df['proposal_to_policy_ratio'] = df['new_policy_count'] / np.maximum(df['unique_proposal'], 1)
    if all(col in df.columns for col in ['new_policy_count', 'unique_quotations']):
        df['quotation_to_policy_ratio'] = df['new_policy_count'] / np.maximum(df['unique_quotations'], 1)

    for col_to_transform in ['unique_proposal', 'unique_quotations', 'unique_customers', 'ANBP_value', 'net_income']:
        if col_to_transform in df.columns:
            df[f'log_{col_to_transform}'] = np.log1p(df[col_to_transform])
            df[f'sqrt_{col_to_transform}'] = np.sqrt(np.maximum(0, df[col_to_transform])) # ensure non-negative for sqrt

print("Creating enhanced historical agent features...")
train_hist_features = pd.DataFrame()
test_hist_features = pd.DataFrame()
hist_data_list_train = []
for agent in train_df['agent_code'].unique():
    agent_data = train_df[train_df['agent_code'] == agent].copy().sort_values('year_month')
    for i in range(1, len(agent_data)):
        past_data = agent_data.iloc[:i]
        current_row_id = agent_data.iloc[i]['row_id']
        hist_data = {'row_id': current_row_id}
        if not past_data.empty:
            hist_data.update({
                'hist_avg_proposals': past_data['unique_proposal'].mean(),
                'hist_avg_quotations': past_data['unique_quotations'].mean(),
                'hist_avg_customers': past_data['unique_customers'].mean(),
                'hist_avg_policies': past_data['new_policy_count'].mean() if 'new_policy_count' in past_data.columns else 0,
                'hist_months_active': len(past_data),
                'hist_zero_policy_months': (past_data['new_policy_count'] == 0).sum() if 'new_policy_count' in past_data.columns else 0,
                'hist_nill_rate': (past_data['new_policy_count'] == 0).mean() if 'new_policy_count' in past_data.columns and len(past_data) > 0 else 0.5,
            })
            if len(past_data) >= 3:
                recent_data = past_data.tail(3)
                hist_data.update({
                    'hist_recent_vs_all_proposals': recent_data['unique_proposal'].mean() / np.maximum(past_data['unique_proposal'].mean(), 1),
                    'hist_recent_vs_all_quotations': recent_data['unique_quotations'].mean() / np.maximum(past_data['unique_quotations'].mean(), 1),
                    'hist_recent_vs_all_customers': recent_data['unique_customers'].mean() / np.maximum(past_data['unique_customers'].mean(), 1),
                    'hist_recent_vs_all_policies': recent_data['new_policy_count'].mean() / np.maximum(past_data['new_policy_count'].mean(), 1) if 'new_policy_count' in past_data.columns else 1,
                    'hist_consistency_score': 1 / (1 + (past_data['unique_proposal'].std() / (past_data['unique_proposal'].mean() + 1) +
                                                   past_data['unique_quotations'].std() / (past_data['unique_quotations'].mean() + 1) +
                                                   past_data['unique_customers'].std() / (past_data['unique_customers'].mean() + 1))/3),
                    'hist_policy_consistency': 1 / (1 + past_data['new_policy_count'].std() / (past_data['new_policy_count'].mean() + 1)) if 'new_policy_count' in past_data.columns else 0,
                })
            if i >= 2:
                hist_data.update({
                    'hist_proposal_growth': (agent_data.iloc[i-1]['unique_proposal'] / np.maximum(agent_data.iloc[i-2]['unique_proposal'], 1)) - 1,
                    'hist_quotation_growth': (agent_data.iloc[i-1]['unique_quotations'] / np.maximum(agent_data.iloc[i-2]['unique_quotations'], 1)) - 1,
                    'hist_customer_growth': (agent_data.iloc[i-1]['unique_customers'] / np.maximum(agent_data.iloc[i-2]['unique_customers'], 1)) - 1,
                    'hist_policy_growth': (agent_data.iloc[i-1]['new_policy_count'] / np.maximum(agent_data.iloc[i-2]['new_policy_count'], 1)) - 1 if 'new_policy_count' in agent_data.columns else 0,
                })
        hist_data_list_train.append(hist_data)
if hist_data_list_train: train_hist_features = pd.DataFrame(hist_data_list_train)

hist_data_list_test = []
for agent in test_df['agent_code'].unique():
    agent_train_history = train_df[train_df['agent_code'] == agent].copy()
    agent_test_data = test_df[test_df['agent_code'] == agent].copy()
    if not agent_test_data.empty:
        agent_all_data = pd.concat([agent_train_history, agent_test_data]).sort_values('year_month').reset_index(drop=True)
        for i, test_row in agent_test_data.iterrows():
            test_date = test_row['year_month']
            past_data = agent_all_data[agent_all_data['year_month'] < test_date]
            hist_data = {'row_id': test_row['row_id']}
            if not past_data.empty:
                hist_data.update({
                    'hist_avg_proposals': past_data['unique_proposal'].mean(),
                    'hist_avg_quotations': past_data['unique_quotations'].mean(),
                    'hist_avg_customers': past_data['unique_customers'].mean(),
                    'hist_avg_policies': past_data['new_policy_count'].mean() if 'new_policy_count' in past_data.columns else 0,
                    'hist_months_active': len(past_data),
                    'hist_zero_policy_months': (past_data['new_policy_count'] == 0).sum() if 'new_policy_count' in past_data.columns else 0,
                    'hist_nill_rate': (past_data['new_policy_count'] == 0).mean() if 'new_policy_count' in past_data.columns and len(past_data) > 0 else 0.5,
                })
                if len(past_data) >= 3:
                    recent_data = past_data.tail(3)
                    hist_data.update({
                        'hist_recent_vs_all_proposals': recent_data['unique_proposal'].mean() / np.maximum(past_data['unique_proposal'].mean(), 1),
                        'hist_recent_vs_all_quotations': recent_data['unique_quotations'].mean() / np.maximum(past_data['unique_quotations'].mean(), 1),
                        'hist_recent_vs_all_customers': recent_data['unique_customers'].mean() / np.maximum(past_data['unique_customers'].mean(), 1),
                        'hist_recent_vs_all_policies': recent_data['new_policy_count'].mean() / np.maximum(past_data['new_policy_count'].mean(), 1) if 'new_policy_count' in past_data.columns else 1,
                        'hist_consistency_score': 1 / (1 + (past_data['unique_proposal'].std() / (past_data['unique_proposal'].mean() + 1) +
                                                       past_data['unique_quotations'].std() / (past_data['unique_quotations'].mean() + 1) +
                                                       past_data['unique_customers'].std() / (past_data['unique_customers'].mean() + 1))/3),
                        'hist_policy_consistency': 1 / (1 + past_data['new_policy_count'].std() / (past_data['new_policy_count'].mean() + 1)) if 'new_policy_count' in past_data.columns else 0,
                    })
                if len(past_data) >= 2:
                    last_two = past_data.sort_values('year_month').tail(2)
                    if len(last_two) >= 2:
                        hist_data.update({
                            'hist_proposal_growth': (last_two.iloc[1]['unique_proposal'] / np.maximum(last_two.iloc[0]['unique_proposal'], 1)) - 1,
                            'hist_quotation_growth': (last_two.iloc[1]['unique_quotations'] / np.maximum(last_two.iloc[0]['unique_quotations'], 1)) - 1,
                            'hist_customer_growth': (last_two.iloc[1]['unique_customers'] / np.maximum(last_two.iloc[0]['unique_customers'], 1)) - 1,
                            'hist_policy_growth': (last_two.iloc[1]['new_policy_count'] / np.maximum(last_two.iloc[0]['new_policy_count'], 1)) - 1 if 'new_policy_count' in last_two.columns else 0,
                        })
            else: # No history, default values
                 hist_data.update({
                    'hist_avg_proposals': 0, 'hist_avg_quotations': 0, 'hist_avg_customers': 0, 'hist_avg_policies': 0,
                    'hist_proposal_growth': 0, 'hist_quotation_growth': 0, 'hist_customer_growth': 0, 'hist_policy_growth': 0,
                    'hist_consistency_score': 0.5, 'hist_policy_consistency': 0.5, 'hist_months_active': 0, 'hist_zero_policy_months': 0,
                    'hist_nill_rate': 0.5, 'hist_recent_vs_all_proposals': 1, 'hist_recent_vs_all_quotations': 1,
                    'hist_recent_vs_all_customers': 1, 'hist_recent_vs_all_policies': 1
                })
            hist_data_list_test.append(hist_data)
if hist_data_list_test: test_hist_features = pd.DataFrame(hist_data_list_test)

default_hist_cols = {col: 0 for col in train_hist_features.columns if col != 'row_id'}
default_hist_cols.update({ # Default for specific rates/ratios
    'hist_nill_rate': 0.5, 'hist_recent_vs_all_proposals': 1, 'hist_recent_vs_all_quotations': 1,
    'hist_recent_vs_all_customers': 1, 'hist_recent_vs_all_policies': 1,
    'hist_consistency_score': 0.5, 'hist_policy_consistency': 0.5
})

if not train_hist_features.empty:
    train_hist_features['row_id'] = train_hist_features['row_id'].astype(int)
    train_df = pd.merge(train_df, train_hist_features, on='row_id', how='left')
    train_df.fillna(value=default_hist_cols, inplace=True)

if not test_hist_features.empty:
    test_hist_features['row_id'] = test_hist_features['row_id'].astype(int)
    test_df = pd.merge(test_df, test_hist_features, on='row_id', how='left')
    test_df.fillna(value=default_hist_cols, inplace=True)


print("Adding agent profiling features...")
agent_profiles_agg = {
    'unique_proposal': ['mean', 'std', 'max', 'min'],
    'unique_quotations': ['mean', 'std', 'max', 'min'],
    'unique_customers': ['mean', 'std', 'max', 'min']
}
if 'new_policy_count' in train_df.columns: # Add if exists
    agent_profiles_agg['new_policy_count'] = ['mean', 'std', 'max', 'min']

agent_profiles = train_df.groupby('agent_code').agg(agent_profiles_agg).reset_index()
agent_profiles.columns = ['_'.join(col).strip('_') for col in agent_profiles.columns.values]
agent_profiles.rename(columns={'agent_code_': 'agent_code'}, inplace=True) # Fix agent_code name

agent_profiles['proposal_cv'] = agent_profiles['unique_proposal_std'] / np.maximum(agent_profiles['unique_proposal_mean'], 1)
agent_profiles['quotation_cv'] = agent_profiles['unique_quotations_std'] / np.maximum(agent_profiles['unique_quotations_mean'], 1)
agent_profiles['customer_cv'] = agent_profiles['unique_customers_std'] / np.maximum(agent_profiles['unique_customers_mean'], 1)
if 'new_policy_count_mean' in agent_profiles.columns and 'new_policy_count_std' in agent_profiles.columns:
    agent_profiles['policy_cv'] = agent_profiles['new_policy_count_std'] / np.maximum(agent_profiles['new_policy_count_mean'], 1)

if 'new_policy_count' in train_df.columns:
    agent_nill_rates = train_df.groupby('agent_code')['new_policy_count'].apply(lambda x: (x == 0).mean()).reset_index()
    agent_nill_rates.columns = ['agent_code', 'agent_nill_rate']
    agent_profiles = pd.merge(agent_profiles, agent_nill_rates, on='agent_code', how='left')

train_df = pd.merge(train_df, agent_profiles, on='agent_code', how='left')
test_df = pd.merge(test_df, agent_profiles, on='agent_code', how='left')

# Fill NAs in profile features created by merge (for agents in test but not train)
profile_feature_cols = [col for col in agent_profiles.columns if col != 'agent_code']
for df in [train_df, test_df]:
    for feature in profile_feature_cols:
        if feature in df.columns:
            df[feature] = df[feature].fillna(0) # Fill with 0 or median/mean based on feature nature
            if 'agent_nill_rate' in feature:
                df[feature] = df[feature].fillna(0.5) # Default NILL rate for unseen agents

print(f"Train data shape after feature engineering: {train_df.shape}")
print(f"Test data shape after feature engineering: {test_df.shape}")
if 'row_id' in test_df.columns: # Check if row_id exists before asserting
    assert test_df['row_id'].nunique() == 914, "Test data size changed or duplicates introduced after feature engineering!"


# Advanced feature selection with stability analysis
print("\nStep 4: Feature selection with stability analysis...")
base_features = ['agent_age','agent_age_squared','agent_age_prime','agent_senior','unique_proposal','unique_quotations','unique_customers','unique_proposals_last_7_days','unique_proposals_last_15_days','unique_proposals_last_21_days','unique_quotations_last_7_days','unique_quotations_last_15_days','unique_quotations_last_21_days','unique_customers_last_7_days','unique_customers_last_15_days','unique_customers_last_21_days','ANBP_value','net_income','number_of_policy_holders','number_of_cash_payment_policies']
engineered_features = ['months_with_company','months_to_first_sale','months_since_first_sale','proposal_trend_7_15','proposal_trend_15_21','quotation_trend_7_15','quotation_trend_15_21','customer_trend_7_15','customer_trend_15_21','proposal_variance','proposal_consistency','quotation_variance','quotation_consistency','customer_variance','customer_consistency','proposals_per_customer','quotations_per_customer','quotations_per_proposal','is_quarter_end','is_year_end','year_month_month_sin','year_month_month_cos','quotation_to_proposal_ratio','cash_payment_ratio','age_experience_interaction','age_sales_experience','proposal_velocity','quotation_velocity','customer_velocity','proposal_momentum','quotation_momentum','customer_momentum','proposal_gap','quotation_gap','revenue_per_proposal','revenue_per_customer','log_unique_proposal','log_unique_quotations','log_unique_customers','log_ANBP_value','log_net_income','sqrt_unique_proposal','sqrt_unique_quotations','sqrt_unique_customers','sqrt_ANBP_value','sqrt_net_income']
historical_features = ['hist_avg_proposals','hist_avg_quotations','hist_avg_customers','hist_avg_policies','hist_consistency_score','hist_policy_consistency','hist_proposal_growth','hist_quotation_growth','hist_customer_growth','hist_policy_growth','hist_months_active','hist_zero_policy_months','hist_nill_rate','hist_recent_vs_all_proposals','hist_recent_vs_all_quotations','hist_recent_vs_all_customers','hist_recent_vs_all_policies']
profile_features = [col for col in agent_profiles.columns if col not in ['agent_code']] # Dynamic based on created profiles
all_potential_features = base_features + engineered_features + historical_features + profile_features
features_to_use = sorted(list(set([f for f in all_potential_features if f in train_df.columns and f in test_df.columns]))) # Unique and sorted
print(f"Total potential features initially: {len(features_to_use)}")

X_temp = train_df[features_to_use].copy()
for col in X_temp.columns:
    if X_temp[col].isnull().any(): X_temp[col] = X_temp[col].fillna(X_temp[col].median())
corr_matrix = X_temp.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop_corr = [column for column in upper.columns if any(upper[column] > 0.95)]
features_to_use = [f for f in features_to_use if f not in to_drop_corr]
print(f"Removed {len(to_drop_corr)} highly correlated features. Features remaining: {len(features_to_use)}")

X = train_df[features_to_use].copy()
y = train_df['target_column'].copy()
for col in X.columns:
    if X[col].isnull().any():
        if pd.api.types.is_numeric_dtype(X[col]): X[col] = X[col].fillna(X[col].median())
        else: X[col] = X[col].fillna('unknown').astype(str) # Or mode
X_scaled = StandardScaler().fit_transform(X)

rf_selector = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=RANDOM_STATE, n_jobs=-1, class_weight='balanced')
rf_selector.fit(X_scaled, y)
importances_rf = pd.Series(rf_selector.feature_importances_, index=features_to_use).sort_values(ascending=False)
selected_features_rf = list(importances_rf[importances_rf > importances_rf.quantile(0.25)].index) # Top 75% by value (approx)

xgb_selector = xgb.XGBClassifier(n_estimators=100, learning_rate=0.05, max_depth=4, random_state=RANDOM_STATE, use_label_encoder=False, eval_metric='logloss')
xgb_selector.fit(X_scaled, y)
importances_xgb = pd.Series(xgb_selector.feature_importances_, index=features_to_use).sort_values(ascending=False)
selected_features_xgb = list(importances_xgb[importances_xgb > importances_xgb.quantile(0.25)].index)

final_features = sorted(list(set(selected_features_rf) | set(selected_features_xgb)))
critical_features_to_add = ['proposal_momentum','customer_momentum','hist_nill_rate','hist_zero_policy_months','agent_nill_rate','months_with_company','proposal_consistency']
for feat in critical_features_to_add:
    if feat in features_to_use and feat not in final_features: final_features.append(feat)
final_features = sorted(list(set(final_features))) # Ensure uniqueness and sort
print(f"Final feature count after selection: {len(final_features)}")
with open(os.path.join(output_dir, 'selected_features.txt'), 'w') as f:
    for feature in final_features: f.write(f"{feature}\n")

# Prepare data for Optuna and final model
global_final_X = train_df[final_features].copy()
global_final_y = train_df['target_column'].copy()

for col in global_final_X.columns:
    if global_final_X[col].isnull().any():
        if pd.api.types.is_numeric_dtype(global_final_X[col]):
            global_final_X[col] = global_final_X[col].fillna(global_final_X[col].median())
        else: # Assuming categorical, fill with mode or 'unknown'
            global_final_X[col] = global_final_X[col].fillna(global_final_X[col].mode()[0]).astype(str)


global_final_scaler = StandardScaler()
global_final_X_scaled = global_final_scaler.fit_transform(global_final_X)
global_tscv = TimeSeriesSplit(n_splits=5) # Ensure tscv is defined for objective

# Optuna Objective Function
def objective(trial):
    # RandomForest
    rf_n_estimators = trial.suggest_int('rf_n_estimators', 50, 300, step=50)
    rf_max_depth = trial.suggest_int('rf_max_depth', 4, 12)
    rf_min_samples_split = trial.suggest_int('rf_min_samples_split', 2, 20)
    rf_min_samples_leaf = trial.suggest_int('rf_min_samples_leaf', 1, 10)
    rf_max_features = trial.suggest_categorical('rf_max_features', ['sqrt', 'log2', 0.6, 0.8])

    # GradientBoosting
    gb_n_estimators = trial.suggest_int('gb_n_estimators', 50, 300, step=50)
    gb_learning_rate = trial.suggest_float('gb_learning_rate', 0.01, 0.15, log=True)
    gb_max_depth = trial.suggest_int('gb_max_depth', 3, 7)
    gb_subsample = trial.suggest_float('gb_subsample', 0.7, 1.0)

    # XGBoost
    xgb_n_estimators = trial.suggest_int('xgb_n_estimators', 50, 300, step=50)
    xgb_learning_rate = trial.suggest_float('xgb_learning_rate', 0.01, 0.15, log=True)
    xgb_max_depth = trial.suggest_int('xgb_max_depth', 3, 7)
    xgb_min_child_weight = trial.suggest_int('xgb_min_child_weight', 1, 7)
    xgb_subsample = trial.suggest_float('xgb_subsample', 0.7, 1.0)
    xgb_colsample_bytree = trial.suggest_float('xgb_colsample_bytree', 0.7, 1.0)
    xgb_gamma = trial.suggest_float('xgb_gamma', 0, 0.3)

    # LightGBM
    lgb_n_estimators = trial.suggest_int('lgb_n_estimators', 50, 300, step=50)
    lgb_learning_rate = trial.suggest_float('lgb_learning_rate', 0.01, 0.15, log=True)
    lgb_num_leaves = trial.suggest_int('lgb_num_leaves', 15, 60)
    lgb_max_depth = trial.suggest_int('lgb_max_depth', 3, 7) # Often -1, but can be tuned
    lgb_min_child_samples = trial.suggest_int('lgb_min_child_samples', 10, 40)

    # CatBoost
    cat_iterations = trial.suggest_int('cat_iterations', 50, 300, step=50)
    cat_learning_rate = trial.suggest_float('cat_learning_rate', 0.01, 0.15, log=True)
    cat_depth = trial.suggest_int('cat_depth', 4, 8)
    cat_l2_leaf_reg = trial.suggest_float('cat_l2_leaf_reg', 1.0, 9.0, log=True)
    cat_class_weight_0 = trial.suggest_float('cat_class_weight_0_cb', 1.5, 4.5) # Tuned class weight for NILL

    # Ensemble Weights
    w_rf = trial.suggest_float('w_rf', 0.5, 2.5)
    w_gb = trial.suggest_float('w_gb', 0.5, 2.5)
    w_xgb = trial.suggest_float('w_xgb', 0.5, 2.5)
    w_lgb = trial.suggest_float('w_lgb', 0.5, 2.5)
    w_cat = trial.suggest_float('w_cat', 0.5, 2.5)
    ensemble_weights = [w_rf, w_gb, w_xgb, w_lgb, w_cat]

    fold_roc_auc_scores = []
    for fold, (train_idx, val_idx) in enumerate(global_tscv.split(global_final_X_scaled)):
        X_train, X_val = global_final_X_scaled[train_idx], global_final_X_scaled[val_idx]
        y_train, y_val = global_final_y.iloc[train_idx], global_final_y.iloc[val_idx]

        rf = RandomForestClassifier(n_estimators=rf_n_estimators, max_depth=rf_max_depth, min_samples_split=rf_min_samples_split, min_samples_leaf=rf_min_samples_leaf, max_features=rf_max_features, random_state=RANDOM_STATE, class_weight='balanced', n_jobs=-1)
        gb = GradientBoostingClassifier(n_estimators=gb_n_estimators, learning_rate=gb_learning_rate, max_depth=gb_max_depth, subsample=gb_subsample, random_state=RANDOM_STATE)

        pos_weight_fold = (y_train == 0).sum() / max(1, (y_train == 1).sum())
        xgb_m = xgb.XGBClassifier(n_estimators=xgb_n_estimators, learning_rate=xgb_learning_rate, max_depth=xgb_max_depth, min_child_weight=xgb_min_child_weight, subsample=xgb_subsample, colsample_bytree=xgb_colsample_bytree, gamma=xgb_gamma, scale_pos_weight=pos_weight_fold, random_state=RANDOM_STATE, use_label_encoder=False, eval_metric='logloss')

        lgb_m = lgb.LGBMClassifier(n_estimators=lgb_n_estimators, learning_rate=lgb_learning_rate, num_leaves=lgb_num_leaves, max_depth=lgb_max_depth, min_child_samples=lgb_min_child_samples, random_state=RANDOM_STATE, class_weight='balanced', verbose=-1)
        cat_m = cb.CatBoostClassifier(iterations=cat_iterations, learning_rate=cat_learning_rate, depth=cat_depth, l2_leaf_reg=cat_l2_leaf_reg, random_seed=RANDOM_STATE, loss_function='Logloss', verbose=0, class_weights={0: cat_class_weight_0, 1: 1.0})

        ensemble = VotingClassifier(estimators=[('rf', rf), ('gb', gb), ('xgb', xgb_m), ('lgb', lgb_m), ('cat', cat_m)], voting='soft', weights=ensemble_weights)

        try:
            ensemble.fit(X_train, y_train)
            y_val_proba = ensemble.predict_proba(X_val)[:, 1]
            fold_roc_auc_scores.append(roc_auc_score(y_val, y_val_proba))
        except Exception as e:
            print(f"Trial {trial.number}, Fold {fold+1} error: {e}")
            return 0.0 # Penalize failed trials

    avg_roc_auc = np.mean(fold_roc_auc_scores) if fold_roc_auc_scores else 0.0
    return avg_roc_auc

print("\nStep 5: Hyperparameter Optimization with Optuna...")
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
N_OPTUNA_TRIALS = 10 # IMPORTANT: Increase for real HPO (e.g., 50, 100+)
study.optimize(objective, n_trials=N_OPTUNA_TRIALS, timeout=3600*3) # Example: 3 hour timeout

best_params = study.best_params
print(f"\nBest ROC AUC from Optuna: {study.best_value:.4f}")
print("Best hyperparameters found by Optuna:")
for key, value in best_params.items():
    print(f"  {key}: {value}")

# Step 6: Training final model on all data with optimized parameters
print("\nStep 6: Training final model on all data with optimized parameters...")
final_rf = RandomForestClassifier(
    n_estimators=best_params.get('rf_n_estimators', 200), max_depth=best_params.get('rf_max_depth', 8),
    min_samples_split=best_params.get('rf_min_samples_split', 10), min_samples_leaf=best_params.get('rf_min_samples_leaf', 4),
    max_features=best_params.get('rf_max_features', 'sqrt'), random_state=RANDOM_STATE, n_jobs=-1, class_weight='balanced'
)
final_gb = GradientBoostingClassifier(
    n_estimators=best_params.get('gb_n_estimators', 200), learning_rate=best_params.get('gb_learning_rate', 0.05),
    max_depth=best_params.get('gb_max_depth', 4), subsample=best_params.get('gb_subsample', 0.8), random_state=RANDOM_STATE
)
final_pos_weight = (global_final_y == 0).sum() / max(1, (global_final_y == 1).sum())
final_xgb = xgb.XGBClassifier(
    n_estimators=best_params.get('xgb_n_estimators', 200), learning_rate=best_params.get('xgb_learning_rate', 0.05),
    max_depth=best_params.get('xgb_max_depth', 4), min_child_weight=best_params.get('xgb_min_child_weight', 5),
    subsample=best_params.get('xgb_subsample', 0.8), colsample_bytree=best_params.get('xgb_colsample_bytree', 0.8),
    gamma=best_params.get('xgb_gamma', 0.1), scale_pos_weight=final_pos_weight, random_state=RANDOM_STATE, use_label_encoder=False, eval_metric='logloss'
)
final_lgb = lgb.LGBMClassifier(
    n_estimators=best_params.get('lgb_n_estimators', 200), learning_rate=best_params.get('lgb_learning_rate', 0.05),
    num_leaves=best_params.get('lgb_num_leaves', 20), max_depth=best_params.get('lgb_max_depth', 4),
    min_child_samples=best_params.get('lgb_min_child_samples', 20), random_state=RANDOM_STATE, class_weight='balanced', verbose=-1
)
final_cat = cb.CatBoostClassifier(
    iterations=best_params.get('cat_iterations', 200), learning_rate=best_params.get('cat_learning_rate', 0.05),
    depth=best_params.get('cat_depth', 6), l2_leaf_reg=best_params.get('cat_l2_leaf_reg', 3),
    random_seed=RANDOM_STATE, loss_function='Logloss', verbose=0,
    class_weights={0: best_params.get('cat_class_weight_0_cb', 3.0), 1: 1.0}
)
final_ensemble_weights = [
    best_params.get('w_rf', 1.0), best_params.get('w_gb', 1.5), best_params.get('w_xgb', 2.0),
    best_params.get('w_lgb', 1.5), best_params.get('w_cat', 2.5)
]
final_ensemble = VotingClassifier(
    estimators=[('rf', final_rf), ('gb', final_gb), ('xgb', final_xgb), ('lgb', final_lgb), ('cat', final_cat)],
    voting='soft', weights=final_ensemble_weights
)
final_ensemble.fit(global_final_X_scaled, global_final_y)

# Step 7: Generating optimized test predictions
print("\nStep 7: Generating optimized test predictions...")
X_test = test_df[final_features].copy()
for col in X_test.columns: # Ensure consistent filling for test data
    if X_test[col].isnull().any():
        if pd.api.types.is_numeric_dtype(X_test[col]):
            # Use median from corresponding TRAIN column to avoid data leakage
             X_test[col] = X_test[col].fillna(global_final_X[col].median())
        else:
            X_test[col] = X_test[col].fillna(global_final_X[col].mode()[0]).astype(str)

X_test_scaled = global_final_scaler.transform(X_test) # Use the scaler FIT on train data
test_proba = final_ensemble.predict_proba(X_test_scaled)[:, 1]
assert len(test_proba) == len(test_df), "Prediction length doesn't match test set!"

print("\nGenerating submissions with dynamic thresholds...")
def get_dynamic_threshold(agent_row):
    base_threshold = 0.60
    if 'months_with_company' in agent_row and not pd.isna(agent_row['months_with_company']):
        if agent_row['months_with_company'] <= 3: base_threshold -= 0.04
        elif agent_row['months_with_company'] >= 24: base_threshold += 0.02
    if 'hist_nill_rate' in agent_row and not pd.isna(agent_row['hist_nill_rate']):
        if agent_row['hist_nill_rate'] > 0.5: base_threshold += 0.02
        elif agent_row['hist_nill_rate'] < 0.1: base_threshold -= 0.02
    if 'agent_age' in agent_row and not pd.isna(agent_row['agent_age']):
        if 35 <= agent_row['agent_age'] <= 45: base_threshold -= 0.01
        elif agent_row['agent_age'] < 25 or agent_row['agent_age'] > 55: base_threshold += 0.01
    return max(0.5, min(base_threshold, 0.7))

test_df['dynamic_threshold'] = test_df.apply(get_dynamic_threshold, axis=1)
dynamic_predictions = (test_proba >= test_df['dynamic_threshold']).astype(int)
dynamic_submission = submission_template.copy()
dynamic_submission['target_column'] = dynamic_predictions
dynamic_submission_path = os.path.join(output_dir, 'dynamic_submission_optuna.csv')
dynamic_submission.to_csv(dynamic_submission_path, index=False)

print("\nGenerating submissions with fixed thresholds:")
thresholds_to_try = np.arange(0.55, 0.66, 0.01)
for threshold_val in thresholds_to_try:
    test_predictions_fixed = (test_proba >= threshold_val).astype(int)
    submission_fixed = submission_template.copy()
    submission_fixed['target_column'] = test_predictions_fixed
    submission_path_fixed = os.path.join(output_dir, f'submission_threshold_{threshold_val:.2f}_optuna.csv')
    submission_fixed.to_csv(submission_path_fixed, index=False)
    print(f"Threshold {threshold_val:.2f}: {test_predictions_fixed.sum()} non-NILL, {len(test_predictions_fixed)-test_predictions_fixed.sum()} NILL")

best_fixed_threshold = 0.60 # Based on prior knowledge or analysis
optimal_predictions = (test_proba >= best_fixed_threshold).astype(int)
optimal_submission = submission_template.copy()
optimal_submission['target_column'] = optimal_predictions
optimal_submission_path = os.path.join(output_dir, 'submission_optuna.csv') # Main submission
optimal_submission.to_csv(optimal_submission_path, index=False)
print(f"\nOptimal submission file created: {optimal_submission_path}")
print(f"Optimal prediction counts (fixed threshold {best_fixed_threshold}): {pd.Series(optimal_predictions).value_counts(normalize=True)}")

model_path = os.path.join(output_dir, 'champion_model_ensemble_optuna.pkl')
joblib.dump((global_final_scaler, final_ensemble, final_features, best_params), model_path)
np.save(os.path.join(output_dir, 'test_probabilities_optuna.npy'), test_proba)
test_df[['row_id', 'agent_code', 'dynamic_threshold']].to_csv(os.path.join(output_dir, 'dynamic_thresholds_optuna.csv'), index=False)

# Step 8: Feature importance analysis
print("\nStep 8: Feature importance analysis (from Optuna-tuned model)...")
if hasattr(final_ensemble, 'named_estimators_'):
    importances_data = {'Feature': final_features}
    if 'rf' in final_ensemble.named_estimators_ and hasattr(final_ensemble.named_estimators_['rf'], 'feature_importances_'):
        importances_data['RF_Importance'] = final_ensemble.named_estimators_['rf'].feature_importances_
    if 'xgb' in final_ensemble.named_estimators_ and hasattr(final_ensemble.named_estimators_['xgb'], 'feature_importances_'):
        importances_data['XGB_Importance'] = final_ensemble.named_estimators_['xgb'].feature_importances_
    if 'lgb' in final_ensemble.named_estimators_ and hasattr(final_ensemble.named_estimators_['lgb'], 'feature_importances_'):
        importances_data['LGB_Importance'] = final_ensemble.named_estimators_['lgb'].feature_importances_
    if 'cat' in final_ensemble.named_estimators_ and hasattr(final_ensemble.named_estimators_['cat'], 'feature_importances_'):
        importances_data['CAT_Importance'] = final_ensemble.named_estimators_['cat'].feature_importances_

    feature_importance_df = pd.DataFrame(importances_data)
    importance_cols = [col for col in feature_importance_df.columns if '_Importance' in col]
    if importance_cols: # Only if any importance columns were added
        feature_importance_df['Avg_Importance'] = feature_importance_df[importance_cols].mean(axis=1)
        feature_importance_df = feature_importance_df.sort_values('Avg_Importance', ascending=False)
        feature_importance_df.to_csv(os.path.join(output_dir, 'feature_importance_optuna.csv'), index=False)

        plt.figure(figsize=(12, 10)) # Adjusted for more features potentially
        sns.barplot(x='Avg_Importance', y='Feature', data=feature_importance_df.head(min(30, len(feature_importance_df)))) # Show top 30 or less
        plt.title('Top Features by Average Importance (Optuna-tuned Model)')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'top_features_optuna.png'))
        print("Feature importance report and plot saved.")
    else:
        print("Could not generate feature importances (models might not have them or ensemble structure changed).")
else:
    print("Final ensemble model does not have 'named_estimators_', cannot extract feature importances directly.")


# Optional: Optuna visualization (run these in a Jupyter notebook or adjust for script execution)
# try:
#     fig_history = optuna.visualization.plot_optimization_history(study)
#     fig_history.write_image(os.path.join(output_dir, "optuna_optimization_history.png"))
#     fig_importance = optuna.visualization.plot_param_importances(study)
#     fig_importance.write_image(os.path.join(output_dir, "optuna_param_importances.png"))
#     # fig_slice = optuna.visualization.plot_slice(study, params=['xgb_learning_rate', 'xgb_max_depth'])
#     # fig_slice.write_image(os.path.join(output_dir,"optuna_slice_plot.png"))
#     print("Optuna visualization plots saved (if plotly and kaleido are installed).")
# except Exception as e:
#     print(f"Could not save Optuna plots: {e}. Ensure plotly and kaleido are installed.")


end_time = time.time()
elapsed_time = end_time - start_time
print("\n" + "=" * 100)
print(f"CHAMPIONSHIP SOLUTION WITH OPTUNA HPO completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total execution time: {elapsed_time:.2f} seconds ({elapsed_time/60:.2f} minutes)")
print(f"OPTIMAL SUBMISSION (Optuna): {optimal_submission_path}")
print(f"DYNAMIC SUBMISSION (Optuna): {dynamic_submission_path}")
print("=" * 100)
print("\nKey insights for presentation:")
print("1. Agent activity consistency is more predictive than raw activity volume")
print("2. Historical NILL rates are strong predictors of future performance")
print("3. New agents need different intervention strategies than experienced ones")
print("4. Agent age and months with company have significant interaction effects")
print("5. Momentum features (recent vs historical activity) are critical early warning signs")
print("6. Optuna helped fine-tune model hyperparameters and ensemble weights for potentially improved generalization.")
print("=" * 100)

/home/randitha/Desktop/IT/Personal/DataStormV6/data-storm/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CHAMPIONSHIP KAGGLE SOLUTION - ADVANCED ENSEMBLE WITH CUSTOM AGENT PROFILING & OPTUNA HPO
Starting at: 2025-05-07 17:47:36

Step 1: Loading data with enhanced checks...
Train data shape: (15308, 23)
Test data shape: (914, 23)
Submission template shape: (914, 2)
Performing data integrity checks...

Step 2: Enhanced preprocessing with domain expertise...
Processed training data shape: (14403, 24)
Target distribution: target_column
1    12969
0     1434
Name: count, dtype: int64
Class weights for imbalance handling: {0: np.float64(5.0219665271966525), 1: np.float64(0.5552856812398798)}

Step 3: Advanced feature engineering with agent profiling...
Creating enhanced historical agent features...
Adding agent profiling features...
Train data shape after feature engineering: (14403, 126)
Test data shape after feature engineering: (914, 125)

Step 4: Feature selection with stability analysis...
Total potential features initially: 104
Removed 10 highly correlated features. Features remaining: 94

[I 2025-05-07 17:47:55,044] A new study created in memory with name: no-name-402ebd02-deb3-487b-b67a-0ca2af5aeb29


Final feature count after selection: 84

Step 5: Hyperparameter Optimization with Optuna...


[I 2025-05-07 17:49:34,851] Trial 0 finished with value: 0.7949309605427345 and parameters: {'rf_n_estimators': 150, 'rf_max_depth': 12, 'rf_min_samples_split': 15, 'rf_min_samples_leaf': 6, 'rf_max_features': 0.8, 'gb_n_estimators': 200, 'gb_learning_rate': 0.06803900745073706, 'gb_max_depth': 3, 'gb_subsample': 0.9909729556485982, 'xgb_n_estimators': 250, 'xgb_learning_rate': 0.01777174904859463, 'xgb_max_depth': 3, 'xgb_min_child_weight': 2, 'xgb_subsample': 0.7912726728878613, 'xgb_colsample_bytree': 0.8574269294896714, 'xgb_gamma': 0.12958350559263473, 'lgb_n_estimators': 100, 'lgb_learning_rate': 0.05243180891902853, 'lgb_num_leaves': 21, 'lgb_max_depth': 4, 'lgb_min_child_samples': 21, 'cat_iterations': 150, 'cat_learning_rate': 0.0838375512850209, 'cat_depth': 4, 'cat_l2_leaf_reg': 3.0953114978934915, 'cat_class_weight_0_cb': 3.2772437065861273, 'w_rf': 0.5929008254399954, 'w_gb': 1.7150897038028767, 'w_xgb': 0.8410482473745831, 'w_lgb': 0.630103185970559, 'w_cat': 2.3977710745

KeyboardInterrupt: 